# ПРОЕКТ

### Часть 1. Парсинг датасета


Импортируем необходимые библиотеки

In [5]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import matplotlib as plt
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from selenium.webdriver.common.by import By

Мы ходим получить датасет с успеваемостью учеников по трем школьным предеметам в зависимости от разных факторов. Передаем ссылку на сайт (а теперь попытаемся получить ссылку на нужный датасет, чтобы наш парсинг выглядил немного мощнее)

In [18]:
t_url = 'https://www.kaggle.com/datasets' 
print(t_url) 

https://www.kaggle.com/datasets


Проверяем, нормально ли сайт относится к парсингу

In [19]:
response = requests.get(t_url)
response

<Response [200]>

Все хорошо, значит теперь получаем ссылку через селениум

In [20]:
browser = webdriver.Chrome()
browser.get(t_url)

Теперь мы видим что датасетов очень много. Там есть строка с поиском и фильтром поэтому попробуем поискать нужный датасет в разделе образование. Тогда нажимаем на кнопку education в фильтре

In [21]:
browser.find_element(By.XPATH, '//*[@id="site-content"]/div[5]/div/div/div[2]/button[3]').click()

Действительно, теперь мы видим намного меньше датасетов и все они посвящены образованию. Тогда просто найдем нужный по названию, чтобы получить его ссылку

In [22]:
dataset_url = ''
tree = BeautifulSoup(browser.page_source)

for i in tree.find_all('li', {'aria-label' : 'Student performance prediction'}):
    dataset_url = 'https://www.kaggle.com' + i.a.get('href')
    
dataset_url

''

In [29]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

Ничего не получилось. И это не удивительно, потому что на первой странице нет нашего датасета. Изначально здесь должен был быть цикл, который проходится по страницая до тех пор, пока не найдет нужную ссылку. Но селениум не позврлил реализовать план, зато в процессе я узнала. что датасет на 4 странице. Тогда просто перейдем на нее сразу через кнопку

In [36]:
browser.find_element(By.XPATH, '//*[@id="site-content"]/div[6]/div/div/div/div[3]/p[4]').click()

Вот он перешел на нужную страницу. Теперь достаем ссылку

In [ ]:
WebDriverWait(browser, 2).until(EC.invisibility_of_element((By.XPATH, '//*[@id="site-content"]/div[6]/div/div/div/div[3]/p[4]')))
WebDriverWait(browser, 2).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="site-content"]/div[6]/div/div/div/div[3]/p[4]'))).click()

In [37]:
tree = BeautifulSoup(browser.page_source)

for i in tree.find_all('li', {'aria-label' : 'Student performance prediction'}):
    dataset_url = 'https://www.kaggle.com' + i.a.get('href')

In [38]:
dataset_url # это и есть ссылка на наш датасет!

'https://www.kaggle.com/datasets/rkiattisak/student-performance-in-mathematics'

Теперь снова передадим ссылку драйверу

In [52]:
browser.get(dataset_url)

In [53]:
# теперь с нормальной ссылкой переходим с помощью BeautifulSoup к более удобному формату
tree = BeautifulSoup(browser.page_source) 
# дерево выглядит громоздко, но без этого нельзя быть уверенным, что страница полностью прогрузилась (иначе в таблице будет 45 наблюдений)
tree 

<html lang="en"><head>
<title>Student performance prediction | Kaggle</title>
<meta charset="utf-8"/>
<meta content="index, follow" name="robots"/>
<meta content="This dataset contains information on the performance of students in mathematics" name="description"/>
<meta content="no-cache" name="turbolinks-cache-control"/>
<meta content="education,tabular,beginner,data analytics,classification" name="keywords"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=5.0, minimum-scale=1.0" name="viewport"/>
<meta content="#008ABC" name="theme-color"/>
<script async="" nonce="" src="https://apis.google.com/_/scs/abc-static/_/js/k=gapi.lb.ru.Skdhe20QJqo.O/m=auth2,client/rt=j/sv=1/d=1/ed=1/rs=AHpOoo8w3S4caNlgf6XV2FIpYkxCwJ5ZuQ/cb=gapi.loaded_0?le=scs"></script><script nonce="" type="text/javascript">
    window["pageRequestStartTime"] = 1686763084596;
    window["pageRequestEndTime"] = 1686763084698;
    window["initialPageLoadStartTime"] = new Date().getTime();
  </script>
<l

###### Чтобы не брать полностью готовый датасет, мы возьмем из итоговой стаблицы только значения, а названия колонок достанем из другой части кода в описании, то есть все будем доставать из разных частей страницы
Создаем списки для данных и названий колонок

In [54]:
data = [] 
names = []

Сначала достаем названия колонок в будущей таблице из описания датасета

In [61]:
name = tree.find_all('div', {'class' : 'sc-dmLtQE bPxorc'})
for i in name:
    names.append(i.find_all('strong')[0].text)
    names.append(i.find_all('strong')[1].text), 
    names.append(i.find_all('strong')[2].text),
    names.append(i.find_all('strong')[3].text),
    names.append(i.find_all('strong')[4].text),
    names.append(i.find_all('strong')[5].text),
    names.append(i.find_all('strong')[6].text),
    names.append(i.find_all('strong')[7].text)
names = [n.replace(":", "") for n in names] # чтобы убрать ":" после каждого названия
names

['Gender',
 'Race/ethnicity',
 'Parental level of education',
 'Lunch',
 'Test preparation course',
 'Math score',
 'Reading score',
 'Writing score']

Теперь сами данные. Так как никаких страниц и вкладок дополнительно открывать не нужно, делем парсинг без функции

In [62]:
strings = tree.find_all('span', {'class' : 'sc-cCYyox sc-cjERFW jxeAHX fWvmCS'}) # Это обращение к отдельной строке таблицы

for item in strings:
    data.append({'1': item.find_all('div')[1].text,
                     '2': item.find_all('div')[2].text,
                     '3': item.find_all('div')[3].text,
                     '4': item.find_all('div')[4].text,
                     '5': item.find_all('div')[5].text,
                     '6': item.find_all('div')[6].text,
                     '7': item.find_all('div')[7].text,
                     '8': item.find_all('div')[8].text})

Запишем все это в таблицу

In [63]:
df = pd.DataFrame(data)
df.columns = names # заменим числа в названии на нормальные названия из names
df

,Gender,Race/ethnicity,Parental level of education,Lunch,Test preparation course,Math score,Reading score,Writing score
0,female,group D,some college,standard,completed,59,70,78
1,male,group D,associate's degree,standard,none,96,93,87
2,female,group D,some college,free/reduced,none,57,76,77
3,male,group B,some college,free/reduced,none,70,70,63
4,female,group D,associate's degree,standard,none,83,85,86
...,...,...,...,...,...,...,...,...
1995,male,group C,some college,standard,none,77,77,71
1996,male,group C,some college,standard,none,80,66,66
1997,female,group A,high school,standard,completed,67,86,86
1998,male,group E,high school,standard,none,80,72,62


Осталось записать все это в файл

In [64]:
df.to_csv('AnDan_project.csv', index=False,  encoding='utf-8')

Теперь часть с парсингом закончена. Можем перейти к описанию датасета, визуализации и модели